These packages allow us to reshape and analyze the data

In [74]:
import pandas as pd
import numpy as np

This data comes from here: https://www.phoenixopendata.com/dataset/crime-data

In [55]:
df = pd.read_csv('crimestat.csv')

C:\Users\Sterling\AnacondaNew\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


This gives us a preview of the crime data

In [56]:
df.head()

,INC NUMBER,OCCURRED ON,OCCURRED TO,UCR CRIME CATEGORY,100 BLOCK ADDR,ZIP,PREMISE TYPE,GRID
0,201600000594484,11/01/2015 00:00,NaN,RAPE,13XX E ALMERIA RD,85006.0,SINGLE FAMILY HOUSE,BD30
1,201500002102327,11/01/2015 00:00,11/01/2015 09:00,LARCENY-THEFT,51XX N 15TH ST,85014.0,APARTMENT,BJ30
2,201500002168686,11/01/2015 00:00,11/11/2015 09:30,LARCENY-THEFT,14XX E HIGHLAND AVE,85014.0,PARKING LOT,BI30
3,201500002102668,11/01/2015 00:00,11/01/2015 11:50,MOTOR VEHICLE THEFT,69XX W WOOD ST,85043.0,SINGLE FAMILY HOUSE,AF12
4,201600000052855,11/01/2015 00:00,01/09/2016 00:00,MOTOR VEHICLE THEFT,N 43RD AVE & W CACTUS RD,85029.0,SINGLE FAMILY HOUSE,DA19


This is a grouping of crimes by type of location, with percentages showing the share of crime each location type receives 

In [57]:
df['PREMISE TYPE'].value_counts(ascending=False, normalize=True).head(20)

SINGLE FAMILY HOUSE                    0.148637
APARTMENT                              0.130296
PARKING LOT                            0.085750
STREET / ROADWAY / ALLEY / SIDEWALK    0.061149
RETAIL BUSINESS                        0.052856
STREET / ROADWAY / SIDEWALK            0.050149
VEHICLE                                0.050096
DEPARTMENT / DISCOUNT STORE            0.046964
CONVENIENCE MARKET / STORE             0.042216
DRIVEWAY                               0.038218
GROCERY / SUPER MARKET                 0.023809
SPECIALTY STORE                        0.020909
OTHER                                  0.016303
HOTEL / MOTEL                          0.016009
OFFICE / COMMERCIAL BUILDING           0.015732
GAS / SERVICE STATION                  0.013216
DRUG STORE                             0.012382
FENCED COMMERCIAL YARD                 0.012206
CONDO / TOWNHOUSE                      0.011605
RESTAURANT                             0.010951
Name: PREMISE TYPE, dtype: float64

This is some data manipulation in Pandas that tells us which crime category is the most prevalent in each ZIP code

In [58]:
top_crime_zip = df.groupby('ZIP')['UCR CRIME CATEGORY'].apply(lambda x: x.value_counts().index[0]).reset_index()
top_crime_zip.columns = ['ZIP','Top Crime']
top_crime_zip.sample(15)

,ZIP,Top Crime
74,85266.0,LARCENY-THEFT
50,85210.0,RAPE
11,85017.0,LARCENY-THEFT
95,85326.0,LARCENY-THEFT
61,85248.0,LARCENY-THEFT
17,85023.0,LARCENY-THEFT
103,85351.0,LARCENY-THEFT
31,85043.0,LARCENY-THEFT
10,85016.0,LARCENY-THEFT
6,85012.0,LARCENY-THEFT


This section converts the timestamps into datetime and then groups the crimes by ZIP code and week start date for a weekly count by ZIP

In [59]:
df['time'] = pd.to_datetime(df['OCCURRED ON'], format='%m/%d/%Y  %H:%M')

In [61]:
dates_agg = df.groupby([df['ZIP'],df['time'].dt.to_period('W').dt.start_time])['UCR CRIME CATEGORY'].count().reset_index()

In [63]:
dates_agg.columns = ['ZIP','week_start_date','crime_count']

In [81]:
dates_agg.head()

,ZIP,week_start_date,crime_count
0,85003.0,2015-10-26,1
1,85003.0,2015-11-02,10
2,85003.0,2015-11-09,21
3,85003.0,2015-11-16,18
4,85003.0,2015-11-23,16


This section uses Numpy to calculate a slope of crime time series for each ZIP code

In [88]:
def get_slope(dates,vals):
    x = dates
    y = vals.astype(float)
    seq_x = np.arange(x.size)
    first_fit = np.polyfit(seq_x, y, 1)
    second_fit = np.poly1d(first_fit)
    return second_fit[0]

This section uses Pandas to load a DataFrame with the slope for each ZIP

In [104]:
zc_table = pd.DataFrame(columns=['ZIP', 'slope'])

In [105]:
counter = 0
for zc in dates_agg.ZIP.unique():
    df_slice = dates_agg[dates_agg.ZIP==zc]
    try:
        slope = get_slope(df_slice.week_start_date.values,df_slice.crime_count.values)
    except:
        pass
    zc_table.at[counter,'ZIP'] = zc
    zc_table.at[counter,'slope'] = slope
    counter += 1

C:\Users\Sterling\AnacondaNew\lib\site-packages\numpy\lib\polynomial.py:628: RuntimeWarning: invalid value encountered in true_divide
  lhs /= scale
C:\Users\Sterling\AnacondaNew\lib\site-packages\numpy\lib\polynomial.py:628: RuntimeWarning: invalid value encountered in true_divide
  lhs /= scale
C:\Users\Sterling\AnacondaNew\lib\site-packages\numpy\lib\polynomial.py:628: RuntimeWarning: invalid value encountered in true_divide
  lhs /= scale
C:\Users\Sterling\AnacondaNew\lib\site-packages\numpy\lib\polynomial.py:628: RuntimeWarning: invalid value encountered in true_divide
  lhs /= scale
C:\Users\Sterling\AnacondaNew\lib\site-packages\numpy\lib\polynomial.py:628: RuntimeWarning: invalid value encountered in true_divide
  lhs /= scale
C:\Users\Sterling\AnacondaNew\lib\site-packages\numpy\lib\polynomial.py:628: RuntimeWarning: invalid value encountered in true_divide
  lhs /= scale
C:\Users\Sterling\AnacondaNew\lib\site-packages\numpy\lib\polynomial.py:628: RuntimeWarning: invalid value

In [106]:
zc_table.shape

(116, 2)

In [107]:
zc_table.head()

,ZIP,slope
0,85003,14.1061
1,85004,14.0501
2,85006,25.7464
3,85007,18.1755
4,85008,57.5574


This is the ZIP code with the fastest growing crime rate

In [108]:
zc_table[zc_table.slope==zc_table.slope.max()]

,ZIP,slope
9,85015,67.1456


This is the ZIP code with the slowing growing crime rate

In [109]:
zc_table[zc_table.slope==zc_table.slope.min()]

,ZIP,slope
44,85202,0.980237
